In [ ]:
!pip install fuzzywuzzy
!pip install keras_visualizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import re
from fuzzywuzzy import fuzz
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import activations
from difflib import get_close_matches

/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:

en_letters = [' ','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
ru_letters = [' ','а','б','в','г','д','е','ё','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ъ','ы','ь','э','ю','я']

len(ru_letters)

34

In [ ]:
 dic = np.array([["а","a"],
           ["б","b"],
           ["в","v"],
           ["г","g"],
           ["д","d"],
           ["е","e"],
           ["ё","u"],
           ["ж","j"],
           ["з","z"],
           ["и","e"],
           ["й","i"],
           ["к","k"],
           ["л","l"],
           ["м","m"],
           ["н","n"],
           ["о","o"],
           ["п","p"],
           ["р","r"],
           ["с","s"],
           ["т","t"],
           ["у","o"],
           ["ф","f"],
           ["х","ch"],
           ["ц","ts"],
           ["ч","ch"],
           ["ш","sh"],
           ["щ","sh"],
           ["ъ",""],
           ["ы","e"],
           ["э","a"],
           ["ю","u"],
           ["я","y"]])

In [ ]:
ru = list(dic[1:,0])
en = list(dic[1:,1])
en

['b',
 'v',
 'g',
 'd',
 'e',
 'u',
 'j',
 'z',
 'e',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'o',
 'f',
 'ch',
 'ts',
 'ch',
 'sh',
 'sh',
 '',
 'e',
 'a',
 'u',
 'y']

In [ ]:
_ru = [] 
_en = []
for indx,itm in enumerate(ru):
  _ru.append(ord(itm))

for indx,itm in enumerate(en[0:15]):
  _en.append(ord(itm))

_en


[98, 118, 103, 100, 101, 117, 106, 122, 101, 105, 107, 108, 109, 110, 111]

In [ ]:
#tst = "технологии"
#tst = "интернет"
tst = "банк"
tst = "system"
ret = ""
for c in tst:
   indx = en.index(c)
   ret = ret + ru[indx]

ret


'сястем'

In [ ]:
fuzz.ratio(ret, 'система')

77

In [ ]:
data =  pd.read_csv('/content/crosswordsfiltered.csv')

data['ru'] = data['ru'].astype('str')
data['en'] = data['en'].astype('str')
mask = (data['ru'].str.len() == data['en'].str.len())  
data2 =  data.loc[mask]

data.describe()

,ru,en
count,711,711
unique,649,631
top,модель,partner
freq,3,3


In [ ]:

model = keras.models.Sequential([
    # layers.InputLayer(3,),  
    layers.Embedding(input_dim=34,output_dim=3,input_length=3), 
    layers.Flatten(),     
    layers.Dense(81,activation=activations.relu), 
    layers.Dense(27, activation="softmax")  
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 3)              102       
                                                                 
 flatten (Flatten)           (None, 9)                 0         
                                                                 
 dense (Dense)               (None, 81)                810       
                                                                 
 dense_1 (Dense)             (None, 27)                2214      
                                                                 
Total params: 3,126
Trainable params: 3,126
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def prepare_data(lang_data,dic):
   data = []
   for word in lang_data:
      row = [0] * 15
      for indx,ltr in enumerate(word.strip().lower()):
          if ltr in dic:
            n = dic.index(ltr)
            # if n > 0:
            #    row[indx] =  n/len(dic)
            # else:
            row[indx]  = n
            data.append(row)     
      return data


In [ ]:
def prepare_data2(lang_data,dic):
   data = []
   for word in lang_data:
      row = [0] * 3
      chr = word.strip().lower()
      for indx,ltr in enumerate(chr[0:3:1]):
          if ltr in dic:
               row[indx] =   dic.index(ltr)
      data.append(row)
   return data   



In [ ]:
def prepare_data5(lang_data,dic,target_lang_data,target_dic):
   data = []
   target = []
   for index,word in enumerate(lang_data):
      word = word.strip()
      target_word = target_lang_data[index]
      if len(target_word) > len(word) : 
         target_word = target_word[:len(word)]
      elif len(target_word) < len(word) : 
         word = word[:len(target_word)]

      target_word = target_word.strip()
      word = ' '+word.lower()+' '
      target_word = target_word.lower()

      for indx,ltr in enumerate(word):
            if ltr!=' ':
                row = [0] * 3
                for i,ltr in enumerate(word[(indx-1):(indx+2)]):
                    if ltr in dic:
                        row[i] =   dic.index(ltr)
                data.append(row) 
                target.append(target_dic.index(target_word[indx-1]))                    
   return data,target   

In [ ]:
b,c = prepare_data5(data["ru"],ru_letters,data["en"],en_letters)
b

[[0, 19, 6],
 [19, 6, 4],
 [6, 4, 14],
 [4, 14, 6],
 [14, 6, 15],
 [6, 15, 20],
 [15, 20, 0],
 [0, 16, 17],
 [16, 17, 6],
 [17, 6, 18],
 [6, 18, 1],
 [18, 1, 24],
 [1, 24, 10],
 [24, 10, 33],
 [10, 33, 0],
 [0, 10, 12],
 [10, 12, 16],
 [12, 16, 15],
 [16, 15, 0],
 [0, 11, 16],
 [11, 16, 4],
 [16, 4, 21],
 [4, 21, 18],
 [21, 18, 20],
 [18, 20, 0],
 [0, 17, 18],
 [17, 18, 16],
 [18, 16, 22],
 [16, 22, 6],
 [22, 6, 19],
 [6, 19, 19],
 [19, 19, 10],
 [19, 10, 33],
 [10, 33, 0],
 [0, 12, 1],
 [12, 1, 2],
 [1, 2, 6],
 [2, 6, 13],
 [6, 13, 0],
 [0, 12, 1],
 [12, 1, 9],
 [1, 9, 10],
 [9, 10, 15],
 [10, 15, 16],
 [15, 16, 0],
 [0, 12, 1],
 [12, 1, 12],
 [1, 12, 20],
 [12, 20, 21],
 [20, 21, 19],
 [21, 19, 0],
 [0, 12, 6],
 [12, 6, 20],
 [6, 20, 25],
 [20, 25, 21],
 [25, 21, 17],
 [21, 17, 0],
 [0, 12, 10],
 [12, 10, 16],
 [10, 16, 19],
 [16, 19, 12],
 [19, 12, 0],
 [0, 12, 13],
 [12, 13, 10],
 [13, 10, 6],
 [10, 6, 15],
 [6, 15, 20],
 [15, 20, 0],
 [0, 12, 13],
 [12, 13, 10],
 [13, 10, 14],
 [1

In [ ]:
def prepare_data3(lang_data,dic):
   data = []
   for word in lang_data:
     data.append(dic.index(word[1]))
   return data   

In [ ]:
# configration
model_loss = keras.losses.MeanAbsoluteError()
# model.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#model.compile(optimizer='adam', loss=keras.losses.CategoricalCrossentropy())
model.compile(loss="categorical_crossentropy", optimizer= "adam", metrics=['accuracy'])


In [ ]:
X_train = prepare_data2(data["ru"],ru_letters)
Y_train = prepare_data3(data["en"],en_letters)
X_train = np.asarray(X_train)
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=27)
X_train

array([[19,  6,  4],
       [16, 17,  6],
       [10, 12, 16],
       ...,
       [17,  6, 18],
       [14,  6,  5],
       [18,  1, 19]])

In [ ]:
X_train,Y_train = prepare_data5(data["ru"],ru_letters,data["en"],en_letters)
X_train = np.asarray(X_train)
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=27)
X_train

array([[ 0, 19,  6],
       [19,  6,  4],
       [ 6,  4, 14],
       ...,
       [19, 10, 19],
       [10, 19, 20],
       [19, 20,  0]])

In [ ]:
# X_train,Y_train = prepare_data5(data["ru"],ru_letters,data["en"],en_letters)
# ru_letters[X_train[150][1]]
len(Y_train)

4435

In [ ]:
#training
model_history = model.fit(
    x=X_train,
    y=Y_train,
    epochs=1000,
    verbose=1
)

Epoch 1/1000
139/139 [==============================] - 1s 2ms/step - loss: 3.0387 - accuracy: 0.1822
Epoch 2/1000
139/139 [==============================] - 0s 2ms/step - loss: 2.3726 - accuracy: 0.3590
Epoch 3/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.9074 - accuracy: 0.5182
Epoch 4/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.6852 - accuracy: 0.6365
Epoch 5/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.5644 - accuracy: 0.6929
Epoch 6/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.4864 - accuracy: 0.7159
Epoch 7/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.4322 - accuracy: 0.7186
Epoch 8/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.3911 - accuracy: 0.7258
Epoch 9/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.3590 - accuracy: 0.7335
Epoch 10/1000
139/139 [==============================] - 0s 2ms/step - loss: 1.331

In [ ]:
model.evaluate(X_train, Y_train, verbose=1)

139/139 [==============================] - 0s 2ms/step - loss: 0.5301 - accuracy: 0.8298


[0.5300829410552979, 0.8297632336616516]

In [ ]:
yy = "ект"
ret = prepare_data2([yy],ru_letters)
ret

[[6, 12, 20]]

In [ ]:
r = model.predict(ret)
s = list(r[0]).index(max(r[0]))
en_letters[s]
# s = ""
# r[0]
# for ind in r[0]:
#      s+=en_letters[math.ceil(ind)]
# s     

1/1 [==============================] - 0s 122ms/step


'c'

In [ ]:
max(r[0])
y = r[r != max(r[0])]

In [ ]:

list(r[0]).index(max(r[0]))

3

In [ ]:
r = model.predict(ret)
s = ""
r[0]
for ind in r[0]:
     s+=en_letters[round(ind)]
s 

1/1 [==============================] - 0s 56ms/step


'   a                       '

In [ ]:
import math

In [ ]:
data["ru"][0]

'сегмент'

In [ ]:
X_train,Y_train = prepare_data5(["белый"],ru_letters,["galle"],en_letters)

In [ ]:
# галерея , фермер ,фигура,бар,план,порт,флаг,energy,карта,Паника,мотор     ----  белый - belly  ---- факультет ,,журнал
r = model.predict(X_train)
d = ""
for k in r:
  d = d+  en_letters[list(k).index(max(k))]

d     

1/1 [==============================] - 0s 58ms/step


'bllia'

In [ ]:
word = d 
patterns = ['car', 'fish', 'belly', 'boy','girl','gallary','give','land','wind','water','','faculty','wind','winter','home','television','suit','store','journal','neighbor']

# for txt in range(0, 1000000, 1):
#     patterns.append(patterns)

print('matched words:',get_close_matches(word, patterns))


matched words: []
